# 컴포지션과 상속
객체 지향 디자인에서 일반적으로 사용되는 개념: **상속**  

- 상속을 사용할 때는 그 위험성도 염두해 둬야 하는데, 가장 주된 위험은 부모 클래스를 확장하여 새로운 클래스를 만들 때마다 부모와 강력하게 결합된 새로운 클래스가 생긴다는 점  
- 상속 관련 기능 중 가장 많이 사용하는 것은 코드 재사용인데, 단지 부모 클래스에 있는 메서드를 얻을 수 있기 때문에 상속을 하는 것은 좋지 않은 방법  
- 코드를 재사용하는 올바른 방법은 여러 상황에서 동작 가능하고 쉽게 조합할 수 있는 응집력 높은 객체를 사용하는 것

## 상속이 좋은 선택인 경우
새로운 하위 클래스를 만들 때 클래스가 올바르게 정의되었는지 확인하기 위해 상속된 모든 메서드를 실제로 사용할 것인지 생각해 보는 것이 좋음  

대부분의 메서드를 필요로 하지 않고 재정의 하거나 대체해야한다면 다음과 같은 이유로 설계상의 실수라고 볼 수 있음
1. 상위 클래스는 잘 정의된 인터페이스 대신 막연한 정의와 너무 많은 책임을 가짐
1. 하위 클래스는 확장하려고 하는 상위 클래스의 적절한 세분화가 아님

#### 상속을 잘 사용한 예
- public 메서드와 속성 인터페이스를 정의한 컴포넌트에 대해 이 클래스의 기능을 그대로 물려받으면서 추가 기능을 더하려는 경우 or 특정 기능을 수정하려는 경우
- e.g) http.server 패키지에서 BaseHTTPRequestHandler 기본 클래스와 이 기본 인터페이스의 일부를 추가하거나 변경하여 확장하는 SimpleHTTPRequestHandler 하위 클래스
- 인터페이스 정의도 상속의 또 다른 좋은 예
- 예외 역시 상속의 또 다른 예
    - 파이썬의 표준 예외는 Exception에서 파생됨
    - except Exception: 과 같은 일반 구문을 통해 모든 에러를 catch할 수 있게 해줌

## 상속 안티패턴
부모 클래스는 파생 클래스의 공통 정의의 일부가 되므로 클래스의 public 메서드는 부모 클래스가 정의하는 것과 일치해야 함  

**파이썬의 전형적인 안티패턴의 예시**
도메인 문제를 해결하기 위해 적절한 데이터 구조를 만든 후에 이 데이터 구조를 사용하는 객체를 만들지 않고 데이터 구조 자체를 객체로 만드는 경우  

*예제) 여러 고객에게 정책을 적용하는 기능을 가진 보험 관리 시스템*
- 동시에 변경 사항을 고객에게 적용하거나 저장하기 위해 고객 정보를 메모리에 보관 필요
- 새 고객 정보를 기록하고 정책을 변경하거나 일부 데이터를 편집 하는 기능 필요
- 정책 자체가 변경되면 현재 트랜잭션의 고객 모두에게 적용 필요  

특정 고객의 레코드에 상수시간 접근 필요 -> policy_transaction&#91;customer_id&#93;처럼 구현하는 것이 좋아보임 (+ 딕셔너리 타입도 고려 가능)

In [4]:
import collections

class TransactionalPolicy(collections.UserDict):
    """잘못된 상속의 예"""
    def change_in_policy(self, customer_id, **new_policy_data):
        self[customer_id].update(**new_policy_data)

In [9]:
from datetime import datetime

policy = TransactionalPolicy({
    "client001": {
        "fee": 1000.0, 
        "expiration_date": datetime(2020, 1, 3)
    }
})
print(policy["client001"])
policy.change_in_policy("client001", expiration_date=datetime(2020, 1, 4))
print(policy["client001"])

{'fee': 1000.0, 'expiration_date': datetime.datetime(2020, 1, 3, 0, 0)}
{'fee': 1000.0, 'expiration_date': datetime.datetime(2020, 1, 4, 0, 0)}


In [10]:
dir(policy)

['_MutableMapping__marker',
 '__abstractmethods__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'change_in_policy',
 'clear',
 'copy',
 'data',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

위의 예시는 원하는 기능을 수행하지만 불필요한 메서드들이 많이 포함되어 있음  

위의 디자인의 주요 문제점
1. 계층 구조가 잘못됨
    - 기본 클래스에서 새 클래스를 만드는 것은 개념적인 확장을 의미하는데, TransactionaPolicy라는 이름만 보고 사전 타입이라는 것을 알기 어려움
1. 결합력(coupling)에 대한 문제
    - TransactionalPolicy에 pop()이나 items()와 같은 불필요한 메서드가 포함되어있음
    - 불필요한 메서드가 public이므로 사용자는 부작용이 있을 수도 있는 위의 메서드들을 호출할 수 있음
    
**위의 문제들이 바로 구현 객체를 도메인 객체와 혼합할 때 발생하는 문제**  
**확장은 온전히 기본 클래스에 추가되며 보다 특화된 것을 구현할 때만 이루어져야 함.**

올바른 해결책은?? **컴포지션**을 사용하는 것  
- TransactionalPolicy 자체가 사전이 되는 것이 아니라 사전을 활용하는 것  
- 사전을 private 속성에 저장하고 &#95;&#95;getitem&#95;&#95;()으로 사전의 프록시를 만든 후 필요한 public 메서드 추가 구현하기

In [20]:
class TransactionalPolilcy:
    """컴포지션을 사용한 리팩토링 예제"""
    
    def __init__(self, policy_data, **extra_data):
        self._data = {**policy_data, **extra_data}
        
    def change_in_policy(self, customer_id, **new_policy_data):
        self._data[customer_id].update(**new_policy_data)
        
    def __getitem__(self, customer_id):
        return self._data[customer_id]
    
    def __len__(self):
        return len(self._data)

## 파이썬의 다중 상속
파이썬은 다중 상속을 지원함  
부적절하게 사용된 다중 상속은 잘못 구현되면 큰 문제를 초래할 수 있으므로 조심하여 사용해야 함

다중 상속을 올바르게 사용하기 위해서 새로운 패턴(e.g 어댑터 패턴)과 믹스인(mixin)을 사용

### 메서드 결정 순서(MRO)
다중 상속 시 다이아몬드 문제와 같은 제약 조건이 발생할 수 있음  

**다이아몬드 문제** 

&emsp;&emsp;A  
&emsp;B&emsp;&nbsp;C  
&emsp;&emsp;D  

B와 C가 A를 상속하고 D가 B와 C를 상속하는 경우 D의 입장에서 B와 C가 같은 이름의 메서드를 가진 경우 어떤 메서드를 사용해야 할 지 모호해지는 문제

In [22]:
class BaseModule:
    module_name = "top"
    
    def __init__(self, module_name):
        self.name = module_name
    
    def __str__(self):
        return f"{self.module_name}:{self.name}"
        
class BaseModule1(BaseModule):
    module_name = "module-1"
    
class BaseModule2(BaseModule):
    module_name = "module-2"
    
class BaseModule3(BaseModule):
    module_name = "module-3"
    
class ConcreteModuleA12(BaseModule1, BaseModule2):
    """1과 2 확장"""
    
class ConcreteModuleB23(BaseModule2, BaseModule3):
    """2와 3 확장"""

In [23]:
str(ConcreteModuleA12("test"))

'module-1:test'

위의 예제에서 ConcreteModuleA12이 BaseModule1과 BaseModule2를 상속받아 &#95;&#95;str&#95;&#95;를 호출한 결과 두 부모 클래스 중 어떤 메서드가 호출될 지 불명확해 보임
- 결과적으로 충돌이 발생하지 않고 어떠한 메서드가 호출됨
- 이 때 호출되는 메서드를 정의하는 알고리즘이 C3 linearization 또는 MRO임

구체적으로 클래스에게 결정 순서를 물어볼 수도 있음

In [24]:
[cls.__name__ for cls in ConcreteModuleA12.mro()]

['ConcreteModuleA12', 'BaseModule1', 'BaseModule2', 'BaseModule', 'object']

### 믹스인(mixin)
믹스인은 코드를 재사용하기 위해 일반적인 행동을 캡슐화해놓은 기본 클래스  
믹스인 클래스는 그 자체로는 유용하지 않으며 대부분이 클래스에 정의된 메서드나 속성에 의존하기 때문에 이 클래스만 확장해서는 확실히 동작하지 않음  
보통은 다른 클래스와 함께 믹스인 클래스를 다중 상속하여 믹스인에 있는 메서드나 속성을 사용  

**문자열을 받아서 하이픈(&#45;)으로 구분된 값을 반환하는 파서**

In [25]:
class BaseTokenizer:
    def __init__(self, str_token):
        self.str_token = str_token
    
    def __iter__(self):
        yield from self.str_token.split("-")
        
tk = BaseTokenizer("aaaa-bbbb-cccc-dddd-eeee")
list(tk)

['aaaa', 'bbbb', 'cccc', 'dddd', 'eeee']

기본 클래스를 변경하지 않고 값을 대문자로 변환하는 예제

In [28]:
class UpperIterableMixin:
    def __iter__(self):
        return map(str.upper, super().__iter__())

class Tokenizer(UpperIterableMixin, BaseTokenizer):
    pass

tk = Tokenizer("aaaa-bbbb-cccc-dddd-eeee")
list(tk)

['AAAA', 'BBBB', 'CCCC', 'DDDD', 'EEEE']

Tokenizer 클래스는 믹스인을 사용하기 때문에 새로운 코드가 필요 없음  
Tokenizer는 믹스인에서 &#95;&#95;iter&#95;&#95;를 호출하고 다시 super()를 호출하여 다음 클래스 BaseTokenizer에 위임하는데 이 때는 이미 대문자를 전달하기 때문에 원하는 결과를 얻을 수 있음